
# IA pour la Qualité des Exigences 
**Auteur : Bibata Sombié**  
Ce notebook charge un fichier Excel d'exigences et applique des **règles simples**  pour :
- détecter les **ambiguïtés** (ex. *et/ou*, *rapide*, *convivial*, …),
- signaler les **problèmes** (non mesurable, condition non spécifiée, multi-exigences…),
- proposer une **réécriture-type** (gabarits simples).


In [2]:
import re
import pandas as pd
from pathlib import Path

# Chemin local vers ton fichier Excel
DATA_FILE = Path("requirements_sample.xlsx")  # car il est dans le même dossier que ton notebook
assert DATA_FILE.exists(), f"Dataset non trouvé: {DATA_FILE}"

df = pd.read_excel(DATA_FILE)
df.head()


,id,requirement_text,label,issue_type_hint,suggested_rewrite_hint
0,CAPT-001,Le capteur doit mesurer la température entre -...,OK,NaN,NaN
1,UI-010,Le bouton d’arrêt d’urgence doit être rouge et...,OK,NaN,NaN
2,PERF-020,Le système doit traiter un lot de 100 messages...,OK,NaN,NaN
3,ELEC-012,Le convertisseur doit fonctionner pour une ent...,OK,NaN,NaN
4,ENV-004,L’équipement doit résister à des vibrations de...,OK,NaN,NaN


## Règles simples de qualité

In [3]:

VAGUE_TERMS = [
    "rapide", "rapidement", "convivial", "conviviale", "lisible", "suffisant",
    "assez grand", "agréable", "optimal", "performant", "robuste", "léger", "sûr", "sécurisé"
]

SUBJECTIVE_TERMS = ["convivial", "conviviale", "lisible", "clair", "claire", "agréable", "intuitif", "intuitive"]
AND_OR_PATTERN = r"\bet\/?ou\b"
UNIT_NUMBER_PATTERN = r"\b\d+(?:[\.,]\d+)?\s*(?:v|a|w|°c|ms|s|kg|m|cm|mm|db|kbit/s|hz|khz|mhz|g)\b"
NUMBER_ONLY_PATTERN = r"\b\d+(?:[\.,]\d+)?\b(?!\s*(?:v|a|w|°c|ms|s|kg|m|cm|mm|db|kbit/s|hz|khz|mhz|g))"
MULTI_ACTION_PATTERN = r"\b(et|,|;)\b.*\b(démarrer|initialiser|envoyer|activer|désactiver)\b"
UNSPECIFIED_COND_TERMS = ["si un problème survient", "quand c’est nécessaire", "si nécessaire", "au besoin"]

def detect_issues(text: str):
    t = text.lower()
    issues = []

    # and/or ambiguity
    if re.search(AND_OR_PATTERN, t):
        issues.append("AND_OR")

    # vague terms
    if any(term in t for term in VAGUE_TERMS):
        issues.append("VAGUE_TERM")

    # subjective terms
    if any(term in t for term in SUBJECTIVE_TERMS):
        issues.append("SUBJECTIVE")

    # units missing (numbers without units)
    if re.search(NUMBER_ONLY_PATTERN, t) and not re.search(UNIT_NUMBER_PATTERN, t):
        issues.append("UNIT_MISSING")

    # unspecified conditions
    if any(phrase in t for phrase in UNSPECIFIED_COND_TERMS):
        issues.append("UNSPECIFIED_CONDITION")

    # multiple actions (heuristic)
    if re.search(MULTI_ACTION_PATTERN, t):
        issues.append("MULTIPLE_REQUIREMENTS")

    # unmeasurable adverbs
    if "rapidement" in t or "vite" in t:
        issues.append("UNMEASURABLE")

    # unspecific security
    if "protég" in t and ("comment" not in t and "norme" not in t and "iso" not in t and "aes" not in t):
        issues.append("UNSPECIFIC_SECURITY")

    return list(sorted(set(issues)))


## Propositions de réécriture (gabarits)

In [4]:

def suggest_rewrite(text: str, issues: list):
    t = text
    suggestion = None

    if "AND_OR" in issues:
        # Replace "et/ou" by deterministic statement with config clause
        suggestion = t.replace("et/ou", "et").replace("ET/OU", "et")
        suggestion += " (si l’affichage d’un seul paramètre est requis, cela doit être spécifié dans la configuration)."

    if "VAGUE_TERM" in issues or "UNMEASURABLE" in issues:
        # Add measurable placeholder
        suggestion = "Remplacer les termes vagues par des critères mesurables. Exemple : "
        if "rapide" in t or "rapidement" in t or "vite" in t:
            suggestion += "« Le système doit s’éteindre en moins de 5 s »."
        elif "léger" in t:
            suggestion += "« Le dispositif doit peser moins de 3 kg »."
        elif "robuste" in t:
            suggestion += "« Le dispositif doit résister à une chute de 1 m sans dommage fonctionnel »."
        elif "convivial" in t or "lisible" in t or "clair" in t:
            suggestion += "« Le texte doit être lisible à 50 cm avec un contraste ≥ 4.5:1 »."
        else:
            suggestion += "« Remplacer par une valeur numérique et une unité vérifiable »."

    if "SUBJECTIVE" in issues and suggestion is None:
        suggestion = "Remplacer les termes subjectifs par des critères objectifs et testables (valeurs numériques, normes, conditions de test)."

    if "UNIT_MISSING" in issues:
        suggestion = "Ajouter les unités manquantes (ex.: V, A, W, °C, ms, kg, etc.)."

    if "UNSPECIFIED_CONDITION" in issues:
        suggestion = "Spécifier la condition/événement déclencheur mesurable (ex.: « si la température dépasse 80 °C pendant 5 s, la LED clignote à 2 Hz »)."

    if "MULTIPLE_REQUIREMENTS" in issues:
        suggestion = "Scinder en exigences atomiques (une action/objectif par exigence) pour faciliter le test."

    if "UNSPECIFIC_SECURITY" in issues:
        suggestion = "Préciser la mesure de sécurité et/ou la norme (ex.: « chiffrer en AES-256, rotation des clés toutes les 24 h »)."

    return suggestion


## Application au dataset

In [5]:

df_eval = df.copy()
results = []
for _, row in df_eval.iterrows():
    text = str(row['requirement_text'])
    issues = detect_issues(text)
    suggestion = suggest_rewrite(text, issues)
    results.append({
        "id": row['id'],
        "requirement_text": text,
        "issues": ", ".join(issues) if issues else "",
        "suggested_rewrite": suggestion if suggestion else ""
    })

out = pd.DataFrame(results)
out.head(10)


,id,requirement_text,issues,suggested_rewrite
0,CAPT-001,Le capteur doit mesurer la température entre -...,,
1,UI-010,Le bouton d’arrêt d’urgence doit être rouge et...,,
2,PERF-020,Le système doit traiter un lot de 100 messages...,,
3,ELEC-012,Le convertisseur doit fonctionner pour une ent...,,
4,ENV-004,L’équipement doit résister à des vibrations de...,,
5,PERF-001,Le système doit être rapide.,VAGUE_TERM,Remplacer les termes vagues par des critères m...
6,QUAL-003,L’interface doit être conviviale.,"SUBJECTIVE, VAGUE_TERM",Remplacer les termes vagues par des critères m...
7,UI-005,Le texte doit être lisible.,"SUBJECTIVE, VAGUE_TERM",Remplacer les termes vagues par des critères m...
8,QUAL-004,La batterie doit avoir une bonne autonomie.,,
9,QUAL-006,Le logiciel doit être sûr.,VAGUE_TERM,Remplacer les termes vagues par des critères m...


## Indicateurs simples

In [6]:

total = len(out)
flagged = (out['issues'].str.len() > 0).sum()
print(f"Exigences analysées : {total}")
print(f"Exigences avec problème(s) détecté(s) : {flagged} ({flagged/total:.0%})")
out['issues'].value_counts().head(10)


Exigences analysées : 30
Exigences avec problème(s) détecté(s) : 19 (63%)


issues
                            11
VAGUE_TERM                   5
SUBJECTIVE, VAGUE_TERM       4
AND_OR                       2
UNSPECIFIED_CONDITION        2
UNIT_MISSING                 2
UNMEASURABLE, VAGUE_TERM     1
MULTIPLE_REQUIREMENTS        1
UNMEASURABLE                 1
UNSPECIFIC_SECURITY          1
Name: count, dtype: int64

## Export du rapport

In [8]:

report_path = Path('requirements_report.xlsx')
with pd.ExcelWriter(report_path, engine='xlsxwriter') as writer:
    out.to_excel(writer, index=False, sheet_name='report')
    df.to_excel(writer, index=False, sheet_name='input_sample')
report_path


WindowsPath('requirements_report.xlsx')

In [10]:
df = pd.read_excel('requirements_report.xlsx')
df.head(10)

,id,requirement_text,issues,suggested_rewrite
0,CAPT-001,Le capteur doit mesurer la température entre -...,NaN,NaN
1,UI-010,Le bouton d’arrêt d’urgence doit être rouge et...,NaN,NaN
2,PERF-020,Le système doit traiter un lot de 100 messages...,NaN,NaN
3,ELEC-012,Le convertisseur doit fonctionner pour une ent...,NaN,NaN
4,ENV-004,L’équipement doit résister à des vibrations de...,NaN,NaN
5,PERF-001,Le système doit être rapide.,VAGUE_TERM,Remplacer les termes vagues par des critères m...
6,QUAL-003,L’interface doit être conviviale.,"SUBJECTIVE, VAGUE_TERM",Remplacer les termes vagues par des critères m...
7,UI-005,Le texte doit être lisible.,"SUBJECTIVE, VAGUE_TERM",Remplacer les termes vagues par des critères m...
8,QUAL-004,La batterie doit avoir une bonne autonomie.,NaN,NaN
9,QUAL-006,Le logiciel doit être sûr.,VAGUE_TERM,Remplacer les termes vagues par des critères m...
